# First Books

This notebook prepares a data set of book information for a prediction task to try to predict if a new author will publish a second book.  Michael Ekstrand uses it for teaching data science.

## Setup

In [1]:
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns

## Book Statistics

The first step is to compute some book interaction statistics.

Let's load and link:

In [2]:
links = pl.scan_parquet('gr-book-ids.parquet')
ixs = pl.scan_parquet('full/gr-interactions.parquet')
ixs = ixs.join(links, on='book_id')

Now aggregate into statistics:

In [3]:
work_stats = ixs.groupby('work_id').agg([
    # number of add-to-shelf actions
    pl.col('rec_id').count().alias('n_shelves'),
    # number of distinct users who interact with it
    pl.col('user_id').n_unique().alias('n_users'),
    # number of ratings
    pl.col('rating').where(pl.col('rating') > 0).count().alias('n_rates'),
    # mean rating
    pl.col('rating').where(pl.col('rating') > 0).mean().alias('mean_rate'),
    # number of "positive" ratings
    (pl.col('rating') > 2).sum().alias('n_pos_rates'),
])

Link with work year:

In [4]:
book_info = pl.scan_parquet('gr-book-info.parquet')
book_link = pl.scan_parquet('gr-book-link.parquet')
book_info = book_info.join(book_link, on='book_id')
work_year = book_info.groupby('work_id').agg(pl.col('pub_year').min())
work_stats = work_stats.join(work_year, on='work_id')

Now a bit of a detour - we need authors. Let's load those:

In [5]:
book_authors = pl.scan_parquet('gr-book-authors.parquet')

And we want to get the *first* work of each author:

In [21]:
author_works = book_authors.join(book_info, on='book_id').filter(pl.col('pub_year').is_not_null()).sort([
    'pub_year',
    'pub_month',
    'pub_date',
])
author_first_work = author_works.groupby('author_id').agg(pl.col('work_id').first())

Now we only want authors first works that were published since GoodReads started in 2007, and no later than 2012 to give the author time to have a new book before the data runs out in 2017:

In [22]:
first_work_stats = work_stats.join(author_first_work, on='work_id')
first_work_stats = first_work_stats.filter((pl.col('pub_year') >= 2008) & (pl.col('pub_year') <= 2012))

Ok - now we have a table of first-work statistics.  We're going to take those authors, and find out how many total works they have in the data set.

In [23]:
author_nworks = first_work_stats.select(['author_id']).join(author_works, on='author_id').groupby('author_id').agg([
    pl.col('work_id').n_unique().alias('au_nbooks')
])

Join this with the original table:

In [24]:
mb_table = first_work_stats.join(author_nworks, on='author_id')

Finally, we can compute this entire table. How will Polars do it?

In [25]:
print(mb_table.describe_optimized_plan())

  JOIN:
  LEFT PLAN ON: [col("author_id")]
    JOIN:
    LEFT PLAN ON: [col("work_id")]
      JOIN:
      LEFT PLAN ON: [col("work_id")]
        Aggregate
        	[col("rec_id").count().alias("n_shelves"), col("user_id").n_unique().alias("n_users"), col("rating")
FILTER WHERE [(col("rating").cast(Float64)) > (0f64)].count().alias("n_rates"), col("rating")
FILTER WHERE [(col("rating").cast(Float64)) > (0f64)].mean().alias("mean_rate"), [(col("rating").cast(Float64)) > (2f64)].sum().alias("n_pos_rates")] BY [col("work_id")] FROM
        	  FastProjection { columns: ["book_id", "rec_id", "user_id", "rating", "work_id"] }
    JOIN:
    LEFT PLAN ON: [col("book_id")]
      CACHE[id: e7fa982259fb36bf, count: 1]
        PARQUET SCAN full/gr-interactions.parquet
        PROJECT 4/10 COLUMNS
        SELECTION: None
    RIGHT PLAN ON: [col("book_id")]
      CACHE[id: af9362d6d718b00, count: 1]
        PARQUET SCAN gr-book-ids.parquet
        PROJECT 2/5 COLUMNS
        SELECTION: None
    END J

And run it:

In [26]:
mb_table = mb_table.collect()
mb_table

work_id,n_shelves,n_users,n_rates,mean_rate,n_pos_rates,pub_year,author_id,au_nbooks
i32,u32,u32,u32,f32,u32,u16,i32,u32
16467778,456,445,100,4.05,94,2012,6572544,1
6576098,59,59,24,3.208333,22,2009,138320,1
9563227,5,5,1,4.0,1,2010,1323472,1
4827816,11,11,6,3.0,5,2008,364864,1
13116269,2,2,2,4.0,2,2010,4027968,11
6142640,3,3,1,5.0,1,2008,2736304,1
15384712,35,35,10,3.8,8,2011,4650976,1
44796724,1,1,1,3.0,1,2011,13603312,1
16389110,43,43,31,3.870968,26,2008,4876720,2


In [28]:
mb_table.write_csv('author-first-works.csv')

## Author Names

Let's get author names and work titles for some debugging info.

In [30]:
authors = pl.read_parquet('gr-author-info.parquet')

In [31]:
authors = authors.join(mb_table.select('author_id'), on='author_id')

In [32]:
authors.write_csv('afw-author-names.csv')

In [33]:
works = pl.read_parquet('gr-work-info.parquet')
works = works.join(mb_table.select('work_id'), on='work_id')
works.write_csv('afw-work-titles.csv')